<a href="https://colab.research.google.com/github/martinpius/PYTORCH/blob/main/VGG19_architecture_Pytorch_implimentation_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount = True)
try:
  COLAB = True
  import torch
  print(f"You are on CoLaB with torch version: {torch.__version__}")
except Exception as e:
  print(f"{type(e)}: {e}\n>>>please correct {type(e)} and reload")
  COLAB = False
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
def time_fmt(t: float = 123.891)->float:
  h = int(t / (60 * 60))
  m = int(t % (60 * 60) / 60)
  s = int(t % 60)
  return f"{h}: {m:>02}: {s:>05.2f}"
print(f">>>time elapsed: {time_fmt()}")


Mounted at /content/drive
You are on CoLaB with torch version: 1.8.1+cu101
>>>time elapsed: 0: 02: 03.00


In [2]:
#In this notebook we are going to impliment the VGG19. architecture and train it on cifar10 dataset

In [3]:
#The general model architecture for our case: input=(3,32,32) images
#conv3(64x2)-maxpool-conv3(128x2)-maxpool-conv3(256x3)-maxpool-conv3(512x6)-maxpool-fcx3-out

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.transforms import transforms
import torchvision.datasets as datasets
import time, sys
from tqdm import tqdm


In [16]:
#Hyperparameters for the model
batch_size = 64
EPOCHS = 2
learning_rate = 1e-1
num_classes = 10
VGG19c = [64,64,'M',128,128, 'M',256,256,256,'M',512,512,512,'M',512,512,512,'M']

In [6]:
#The model class: Here we impliment vgg19 like architecture(may not be exactly like original vgg19)

In [7]:
class VGG19Net(nn.Module):
  def __init__(self, num_classes = 1000, in_channels = 3):
    super(VGG19Net, self).__init__()
    self.in_channels = in_channels
    self.conv_layers = self.make_conv(VGG19c)
    self.fc = nn.Sequential(nn.Linear(in_features = 512*7*7, out_features = 4096),
                             nn.ReLU(),
                             nn.Dropout(0.5),
                             nn.Linear(in_features = 4096, out_features = 4096),
                             nn.ReLU(),nn.Dropout(0.5),
                             nn.Linear(in_features = 4096, out_features = num_classes))
  def forward(self, x):
    m = nn.Upsample(scale_factor = 7)
    x = m(x)
    x = self.conv_layers(x)
    x = x.reshape(x.shape[0],-1)
    x = self.fc(x)
    return x
  def make_conv(self, architecture):
    layers = []
    in_channels = self.in_channels
    for x in architecture:
      if type(x) == int:
        out_channels = x
        layers+=[nn.Conv2d(in_channels = in_channels,out_channels = out_channels,kernel_size = (3,3), stride = (1,1),padding = (1,1)),nn.BatchNorm2d(x),nn.ReLU()]
        in_channels = x
      elif x == 'M':
        layers+=[nn.MaxPool2d(kernel_size = (2,2), stride = (2,2))]
    return nn.Sequential(*layers)
      
   

In [8]:
#Check the output shape if is the one we intended (batch-size = 64, num_classes = 1000)
rnd_data = torch.randn(10,3,32,32).to(device)
model = VGG19Net(num_classes=10, in_channels = 3).to(device)
print(f"output shape: {model(rnd_data).shape}")

output shape: torch.Size([10, 10])


In [9]:
print(model)

VGG19Net(
  (conv_layers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, kernel_size=(3, 3)

In [12]:
#Due to computation limitation we are going to freeze some layers so as to allow training to run
for layer in model.conv_layers[:-9]:
  layer.trainable = False


In [13]:
#Load CIFAR10 data:
train_data = datasets.CIFAR10(root = '/train_data', train = True, transform = transforms.ToTensor(), download = True)
test_data = datasets.CIFAR10(root = '/test_data', train = False, transform = transforms.ToTensor(), download = True)
train_loader = DataLoader(dataset = train_data, shuffle = True, batch_size = batch_size)
test_loader = DataLoader(dataset = test_data, shuffle = True, batch_size = batch_size)
for idx, (data, target) in enumerate(train_loader):
  print(f"{data.shape}, {target.shape}")
  sys.exit()


Extracting /train_data/cifar-10-python.tar.gz to /train_data



Extracting /test_data/cifar-10-python.tar.gz to /test_data
torch.Size([64, 3, 32, 32]), torch.Size([64])


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [14]:
optimizer = optim.Adam(params = model.parameters(), lr = learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
#We normally do transfer learning using pre-trained weights due to computation cost.

In [ ]:
#Training loop:
tic = time.time()
for epoch in range(EPOCHS):
  print(f"\>>>>training starts for epoch: {epoch + 1}.....")
  for idx, (data, target) in enumerate(tqdm(train_loader)):
    data = data.to(device)
    target = target.to(device)
    preds = model(data)
    train_loss = loss_fn(preds, target)
    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()
def model_eval(loader, model):
  if loader.dataset.train:
    print(f"\>>>Checking for the training data:.....")
  else:
    print(f"Checking for the validation data:.....")
  num_examples = 0
  num_correct = 0
  model.eval()
  with torch.no_grad():
    for x, y in loader:
      x = x.to(device)
      y = y.to(device)
      preds = model(x)
      _,predictions = preds.max(1)
      num_correct+=(predictions == y).sum()
      num_examples+=predictions.size(0)
  model.train()
  return float(num_correct/num_examples)
toc = time.time()
print(f"Train accuracy: {model_eval(train_loader, model):.2f}")
print(f"Validation accuracy: {model_eval(test_loader, model):.2f}")
print(f"time elapse: {time_fmt(toc - tic)}")
